# Final Project 
## Complete Machine Learning Pipeline for MIMIC Classification

### Mariajose Argote


In [ ]:
EDA = True

: 

### Objective
Build a classification pipeline to predict `HOSPITAL_EXPIRE_FLAG` (patient mortality during ICU stay) using the MIMIC-III dataset subset with 20,885 ICU patient observations.

### Success Criteria
- Working, reproducible Jupyter notebook with complete ML pipeline
- Predictions submitted as probabilities (.predict_proba) in CSV format
- Top-tier prediction ranking (within reasonable distance of best performer)
- Ability to defend all modeling decisions during in-person presentation
- Grade target: 9/10 from presentation + up to 1/10 from prediction ranking

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import jinja2

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
# Generate a random seed for reproducibility
# np.random.seed() has a range of [0, 2**32 - 1] for the seed value
print(np.random.randint(0, 2**30))

SEED = 244459055

np.random.seed(SEED)
#540921260


### 1.1 Import MIMIC Data

In [ ]:
# Import data

train = pd.read_csv('MIMIC III dataset HEF/mimic_train_HEF.csv')
test= pd.read_csv('MIMIC III dataset HEF/mimic_test_HEF.csv')


# Display first few rows of the dataset

print(train.info())

train.describe()



In [ ]:
train.head()

### 1.2 Check data types, missing values, distributions

Visual Inspection
* 10% of heart rate and blood pressure columns are missing / blank
* 11% of SYS and DiasBP columns are missing /blank
* 10% missing respiration rate columns
* 12% missing temperature information
* 11% missing SPO2 columns information


In [ ]:
print(test.info())
test.describe()


In [ ]:
# Check train/test column consistency
target_col = "HOSPITAL_EXPIRE_FLAG"
leaky_cols = ['LOS', 'DOD', 'DISCHTIME','DEATHTIME'] #must drop for modeling
id_cols      = ["subject_id", "hadm_id", "icustay_id"]

# Groups 
vital_cols = [  # 24 cols: min/max/mean of each
    
    "HeartRate_Min", "HeartRate_Max", "HeartRate_Mean",
    "SysBP_Min", "SysBP_Max", "SysBP_Mean",
    "DiasBP_Min", "DiasBP_Max", "DiasBP_Mean",
    "MeanBP_Min", "MeanBP_Max", "MeanBP_Mean",
    "RespRate_Min", "RespRate_Max", "RespRate_Mean",
    "TempC_Min", "TempC_Max", "TempC_Mean",
    "SpO2_Min", "SpO2_Max", "SpO2_Mean",
    "Glucose_Mean", "Glucose_Max", "Glucose_Min"
]

temporal_cols = ["DOB", "ADMITTIME", "DISCHTIME", "Diff"]

demographic_cols = ["GENDER", "ADMISSION_TYPE", "INSURANCE",
             "RELIGION", "MARITAL_STATUS", "ETHNICITY"]

clinical_cols = ["DIAGNOSIS", "ICD9_diagnosis", "FIRST_CAREUNIT"]


# Columns to use as features in train
exclude_cols = set([target_col]) | set(leaky_cols) | set(id_cols)
feature_cols = [c for c in train.columns if c not in exclude_cols]

# Working copy and drop giveaways up front
train = train.copy()
train_model = train.drop(columns=leaky_cols, errors="ignore")

test_model = test.drop(columns=leaky_cols, errors="ignore")

print(train_model.shape, test_model.shape)

In [ ]:

# 1) Check that test has all feature columns
missing_in_test = set(feature_cols) - set(test.columns)
extra_in_test   = set(test.columns) - set(feature_cols)
print("Missing in test:", missing_in_test)
print("Extra in test  :", extra_in_test)

# 2) Reorder test columns to match train's feature order
test = test[feature_cols]


#checking for column type consistency
shared = set(train_model.columns) & set(test.columns)
dtype_diff = {
    col: (train_model[col].dtype, test[col].dtype)
    for col in shared
    if train_model[col].dtype != test[col].dtype
}
print("Columns with different dtypes:", dtype_diff)


In [ ]:
#Check missing values - columns

def missing_summary(train):
    n = len(train)
    miss_cnt = train.isna().sum()
    miss_pct = miss_cnt / n
    return (
        pd.DataFrame({
            "n_missing": miss_cnt,
            "pct_missing": miss_pct
        })
        .sort_values("pct_missing", ascending=False)
    )

col_missing = missing_summary(train.drop(columns=id_cols, errors="ignore")
                              .drop(columns=leaky_cols, errors="ignore"))
print(col_missing.head(15))

#Check missing values - rows
row_missing_count = train_model.isna().sum(axis=1)
row_missing_frac  = row_missing_count / train_model.shape[1]

print("Rows with ANY missing:", (row_missing_count > 0).mean())
print("Rows with > 20% missing:", (row_missing_frac > 0.2).mean())
print("Rows with > 50% missing:", (row_missing_frac > 0.5).mean())


In [ ]:
#check if vital signs are missing together in rows

vital_cols_present = [c for c in vital_cols if c in train_model.columns] 

missing_vitals = train_model[vital_cols_present].isna()

# Fraction of rows with at least one missing vital
rows_any_vital_missing  = missing_vitals.any(axis=1)
# Fraction of rows with all vitals missing
rows_all_vitals_missing = missing_vitals.all(axis=1)

print("Frac rows with ANY vital missing   :", rows_any_vital_missing.mean().round(3))
print("Frac rows with ALL vitals missing  :", rows_all_vitals_missing.mean().round(3))

# See patterns of missingness across vitals
pattern_counts = missing_vitals.value_counts()
print(pattern_counts.head(10))


In [ ]:
#exploring those rows where vitals are missing

# Count missing + present vitals per row
THRESH_MISSING_VITALS = 21

def vitals_missing_stats(df, vital_cols):
    n_vitals_missing  = df[vital_cols].isna().sum(axis=1)
    print(f"Rows in with 21 vitals missing vitals:", 
          (n_vitals_missing == THRESH_MISSING_VITALS).sum())


vitals_missing_stats(train_model, vital_cols) 
    




In [ ]:
vital_cols_wo_glucose = [col for col in vital_cols if "Glucose" not in col]
train_model_clean = train_model.dropna(subset=vital_cols_wo_glucose, how="all")

test_model_clean = test_model.dropna(subset=vital_cols_wo_glucose, how="all")


Because vital signs are central to our mortality prediction, I removed ICU stays where ≥ 87.5% of vital-sign summary features were missing (2,183 out of 20,885 stays). For the remaining patients, the plan is to impute the missing vitals using median/mean values within the training set.

In [ ]:
#checking different dataframe sizes

print("train shape:", train.shape)
print("train_model shape:", train_model.shape)
print("train_model_clean shape:", train_model_clean.shape)
print("test_model shape:", test_model.shape)
print("test_model_clean shape:", test_model_clean.shape)

train_model_clean.head()

In [ ]:
#checking missing data by columns after clean drop

col_missing = missing_summary(train_model_clean.drop(columns=id_cols, errors="ignore"))
col_missing.head(20)


### 1.3 Identify and visualize class imbalance in target

88.8% of ICU stays survived in the cleaned training set, this indicates strong class imbalance in the target

In [ ]:
train_model_clean[target_col].value_counts(normalize=True)

#88.8% of ICU stays survived in the cleaned training set


In [ ]:
#plotting target distribution after cleaning

plt.figure(figsize=(4,4))
sns.countplot(x=target_col, data=train_model_clean)
plt.title("Target Distribution: HOSPITAL_EXPIRE_FLAG")
plt.xticks([0, 1], ["Survived (0)", "Died (1)"])
plt.show()


### 1.4 Examine correlations between vital signs

In [ ]:
vitals_present = [c for c in vital_cols if c in train_model_clean.columns]

vital_means = [c for c in vitals_present if c.endswith("_Mean")]

train_model_clean[vital_means].describe()

In [ ]:
if EDA:
    num_features = train_model_clean[feature_cols].select_dtypes(include=np.number).columns.tolist()

    ncols = 3
    nrows = 5                     # fixed grid: 5 rows x 3 cols = 15 plots per figure
    plots_per_fig = nrows * ncols

    # 2. Loop over features in chunks of 15
    for start in range(0, len(num_features), plots_per_fig):
        subset = num_features[start:start + plots_per_fig]

        fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                                 figsize=(15, 5 * nrows))

        # Make axes always a flat 1D array for simple indexing
        axes = np.array(axes).ravel()

        # 3. Draw each boxplot
        for i, feat in enumerate(subset):
            ax = axes[i]

            # Optional: sample rows if dataset is big to speed up plotting
            data_to_plot = train[feat]
            if len(train_model_clean) > 5000:
                data_to_plot = data_to_plot.sample(5000, random_state=0)

            sns.boxplot(y=data_to_plot, ax=ax)
            ax.set_title(feat)

        # 4. Turn off any leftover empty axes
        for j in range(len(subset), len(axes)):
            axes[j].axis("off")

        plt.tight_layout()
        plt.show()


In [ ]:
# Now we analyze the correlation matrix of the numerical columns

if EDA:
    corr = train_model_clean[num_features].corr()
    display(corr.style.background_gradient(cmap='coolwarm', axis=None).format("{:.4f}"))

In [ ]:
#focusing on vital means correlation heatmap

corr = train_model_clean[vital_means].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Between Mean Vital Signs")
plt.show()


Observations:
* Heart rate and respiration rate are moderately positively correlated
* Systolic, Diastolic BP and MeanBP are highly correlated, as expected
* SpO2 and Respiratory Rate are negatively correlated, as expected
* Heart rate and Respiration Rate and TempC are positively correlated
* Glucose is negatively correlated with Temperature and SpO2

In [ ]:
# Mean vitals by survival vs death
grouped_vitals = (
    train_model_clean
    .groupby(target_col)[vital_means]
    .mean()
    .T
)

grouped_vitals


### 1.5 Explore categorical variable distributions

### 2.1 Feature Engineering age at admission, time-based features

In [ ]:
# Create function for age and time features (used later for test data)

def add_age_and_time_features(df):
    df = df.copy()
    # Convert to datetime
    df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')
    df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], errors='coerce')

    df['AGE'] = np.nan
    validmask = df['DOB'].notna() & df['ADMITTIME'].notna()
    if validmask.any():
        dob = df.loc[validmask, 'DOB']
        admit = df.loc[validmask, 'ADMITTIME']

        years = admit.dt.year - dob.dt.year
        hadbirthday = (
            (admit.dt.month > dob.dt.month) |
            ((admit.dt.month == dob.dt.month) & (admit.dt.day >= dob.dt.day))
        )
        ageyears = years - (~hadbirthday).astype(int)
        df.loc[validmask, 'AGE'] = ageyears

    # clamp to [0,120]
    maskinvalid = (df['AGE'] < 0) | (df['AGE'] > 120)
    df.loc[maskinvalid, 'AGE'] = np.nan

    print(df["AGE"].describe())
    missing_mask = df["AGE"].isna()
    n_missing = missing_mask.sum()
    n_total = len(df)
    print(f"Missing AGE: {n_missing} / {n_total} ({n_missing/n_total:.3f})")

    # mortality among rows with missing AGE
    mortality_missing = df.loc[missing_mask, target_col].mean()
    # mortality among rows with known AGE
    mortality_known   = df.loc[~missing_mask, target_col].mean()

    print(f"Mortality (AGE missing): {mortality_missing:.3f}")
    print(f"Mortality (AGE known)  : {mortality_known:.3f}")

    # simple time features
    df['admit_hour'] = df['ADMITTIME'].dt.hour
    df['admit_weekday'] = df['ADMITTIME'].dt.dayofweek
    df['is_weekend'] = df['admit_weekday'].isin([5, 6]).astype(int)

    return df


In [ ]:
#apply function to clean train data and plot age distribution

train_base = add_age_and_time_features(train_model_clean)

print(train_base.shape)

plt.figure(figsize=(6,4))
sns.histplot(train_base["AGE"], bins=40, kde=True)
plt.title("Age Distribution")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
sns.boxplot(x=target_col, y="AGE", data=train_base)
plt.title("Age by Mortality Outcome")
plt.xticks([0, 1], ["Survived", "Died"])
plt.tight_layout()
plt.show()




In [ ]:
for col in demographic_cols:
    if col in train_base.columns:
        print(f"\n=== {col} ===")
        print(train_base[col].value_counts(normalize=True).head(10))

        plt.figure(figsize=(6,4))
        sns.countplot(
            y=col,
            data=train_base,
            order=train_base[col].value_counts().index
        )
        plt.title(f"Distribution of {col}")
        plt.show()


In [ ]:
#checking specific key categories against target
cat_for_target = ["GENDER","ETHNICITY","INSURANCE", "FIRST_CAREUNIT"]

for col in cat_for_target:
    if col in train_base.columns:
        # Mortality rate by category
        rate = (
            train_base
            .groupby(col)[target_col]
            .mean()
            .sort_values(ascending=False)
        )
        print(f"\nMortality rate by {col}:")
        print(rate)

        plt.figure(figsize=(6,4))
        sns.barplot(x=rate.values, y=rate.index)
        plt.title(f"Mortality Rate by {col}")
        plt.xlabel("Mean HOSPITAL_EXPIRE_FLAG")
        plt.ylabel(col)
        plt.tight_layout()
        plt.show()


### Phase 2.2 Diagnoses Extra Data EDA & Feature Engineering

In [ ]:

# Load diagnoses file
MIMIC_diagnoses = pd.read_csv("MIMIC III dataset HEF/extra_data/MIMIC_diagnoses.csv")

MIMIC_diagnoses.head()
MIMIC_diagnoses.info()


#### 2.2.1 Creating diagnoses feature fitting for train data only

In [ ]:

#restrict diagnoses to only those in train_model_clean

def fit_diagnosis_features(train_df, diagnoses_df):
    df = train_df.copy()
    
    # Restrict diagnoses to hadm_id present in train
    diagnoses_train = diagnoses_df[diagnoses_df['HADM_ID'].isin(df['hadm_id'])].copy()

    # Merge labels
    diagnoses_train = diagnoses_train.merge(
        df[['hadm_id', 'HOSPITAL_EXPIRE_FLAG']],
        left_on='HADM_ID', 
        right_on='hadm_id', 
        how='left'
    )

    # Count diagnoses per admission
    diagnoses_counts = diagnoses_train.groupby('hadm_id').size().reset_index(name='diagnosis_count')

    # Mortality per ICD9
    severity = (
        diagnoses_train
        .groupby('ICD9_CODE')['HOSPITAL_EXPIRE_FLAG']
        .agg(['sum', 'count'])
        .rename(columns={'sum': 'n_deaths', 'count': 'n_patients'})
    )
    overall_mort = df['HOSPITAL_EXPIRE_FLAG'].mean()
    alpha = 10

    severity['raw_mortality'] = severity['n_deaths'] / severity['n_patients']
    severity['smoothed_severity'] = (
        (severity['n_patients'] * severity['raw_mortality'] + alpha * overall_mort)
        / (severity['n_patients'] + alpha)
    )

    # Save mapping for later use
    severity_map = severity['smoothed_severity'].to_dict()

    # Attach severity to all diagnoses
    diagnoses_train['severity'] = diagnoses_train['ICD9_CODE'].map(severity_map)
    diagnoses_train['severity'] = diagnoses_train['severity'].fillna(overall_mort)

    # Aggregate per hadm_id
    sev_agg = diagnoses_train.groupby('hadm_id')['severity'].agg(
        avg_diagnosis_severity='mean',
        max_diagnosis_severity='max'
    ).reset_index()  # ← RESET INDEX HERE

    # Primary diagnosis severity
    primary = diagnoses_train[diagnoses_train['SEQ_NUM'] == 1].copy()
    primary = (primary
               .groupby('hadm_id')['severity']
               .first()  # In case multiple primaries (shouldn't happen but safeguard)
               .reset_index()
               .rename(columns={'severity': 'primary_diagnosis_severity'}))

    # merge features back to df
    df = df.merge(diagnoses_counts, on='hadm_id', how='left')
    df = df.merge(sev_agg, on='hadm_id', how='left')
    df = df.merge(primary, on='hadm_id', how='left')

    # Fill missing (no diagnoses)
    df['diagnosis_count'] = df['diagnosis_count'].fillna(0)
    for col in ['avg_diagnosis_severity', 'max_diagnosis_severity', 'primary_diagnosis_severity']:
        df[col] = df[col].fillna(overall_mort)

    # Pack state
    state = {
        'severity_map': severity_map,
        'overall_mort': overall_mort,
        'alpha': alpha,
    }
    return df, state



#### Severity Quick Comments

**Severity** = mortality risk score per ICD9 diagnosis code

**How it's calculated:**
1. For each ICD9 code in the training data, compute the proportion of patients who died
2. Apply Bayesian smoothing to balance observed rates with the overall mortality rate
3. Rare diagnoses (few patients) shrink toward the average; common diagnoses stay close to observed rates

**Why Bayesian smoothing?**
- Prevents extreme estimates: A rare code with 1 patient who died would have raw mortality = 100%, which is unreliable. With smoothing (α=10), that becomes: (1 × 1.0 + 10 × 0.112) / 11 ≈ 0.19 (more conservative). Common codes with 100+ patients are barely affected by the prior

**α = 10 interpretation:**
- Acts like adding 10 "virtual patients" with average mortality to every ICD9 code
- Balances between observed data (for common codes) and prior knowledge (for rare codes)

Each ICD9 code gets a severity score between 0 (low mortality risk) and 1 (high mortality risk)

In [ ]:
# apply to clean train data
train_base_w_severity, diag_state = fit_diagnosis_features(train_base, MIMIC_diagnoses)



In [ ]:
print(train_base_w_severity.shape, train_model_clean.shape)

#### 2.2.2 Function to apply diagnoses severity features to test data later on

In [ ]:
## apply to test data function

def apply_diagnosis_features(df, diagnoses_df, state):
    df = df.copy()
    severity_map = state['severity_map']
    overall_mort = state['overall_mort']

    diag = diagnoses_df[diagnoses_df['HADM_ID'].isin(df['hadm_id'])].copy()

    diag['severity'] = diag['ICD9_CODE'].map(severity_map)
    diag['severity'] = diag['severity'].fillna(overall_mort)

    diag_counts = diag.groupby('hadm_id').size().rename('diagnosis_count')
    sev_agg = diag.groupby('hadm_id')['severity'].agg(
        avg_diagnosis_severity='mean',
        max_diagnosis_severity='max'
    )
    primary = diag[diag['SEQ_NUM'] == 1].copy()
    primary = primary.set_index('hadm_id')['severity'].rename('primary_diagnosis_severity')

    df = df.join(diag_counts, on='hadm_id')
    df = df.join(sev_agg, on='hadm_id')
    df = df.join(primary, on='hadm_id')

    df['diagnosis_count'] = df['diagnosis_count'].fillna(0)
    for col in ['avg_diagnosis_severity', 'max_diagnosis_severity', 'primary_diagnosis_severity']:
        df[col] = df[col].fillna(overall_mort)

    return df


#### 2.2.3 Distribution of diagnoses per patient at admission

In [ ]:
# Distribution of Diagnoses per Patient

# Filter diagnoses to training admissions only
train_hadm = set(train_base_w_severity['hadm_id'])
diag_train = MIMIC_diagnoses[MIMIC_diagnoses['HADM_ID'].isin(train_hadm)].copy()

# Count diagnoses per admission
diag_per_patient = diag_train.groupby('HADM_ID').size()

print("=== Diagnoses per ICU Stay (Training Data) ===\n")
print(diag_per_patient.describe())
print(f"\nMedian: {diag_per_patient.median():.0f} diagnoses")
print(f"Range: {diag_per_patient.min():.0f} to {diag_per_patient.max():.0f} diagnoses")

# Plot distribution
plt.figure(figsize=(10, 6))
plt.hist(diag_per_patient, bins=40, edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Number of Diagnoses per ICU Stay', fontsize=12)
plt.ylabel('Frequency (Number of ICU Stays)', fontsize=12)
plt.title('Distribution of Diagnosis Count per ICU Stay', fontsize=14, fontweight='bold')
plt.axvline(diag_per_patient.median(), color='red', linestyle='--', linewidth=2,
            label=f'Median: {diag_per_patient.median():.0f}')
plt.axvline(diag_per_patient.mean(), color='orange', linestyle='--', linewidth=2,
            label=f'Mean: {diag_per_patient.mean():.1f}')
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


##### Observation: Diagnosis Volume

- **Median:** ~14 diagnoses per ICU stay
- **Range:** 1 to 39 diagnoses
- **Distribution:** Right-skewed; most patients have 10-20 diagnoses
- **Interpretation:** 
  - Patients with very few diagnoses may represent simpler cases, shorter stays
  - Patients with many diagnoses likely have complex, multi-system conditions
  - Diagnosis count is likely a good proxy for patient complexity/severity

#### 2.2.4 Most Common Primary Diagnoses

In [ ]:
# Most Common Primary Diagnoses

# Extract primary diagnoses (SEQ_NUM == 1)
primary_diag = diag_train[diag_train['SEQ_NUM'] == 1].copy()

# Top 20 most frequent primary ICD9 codes
top_20_primary = primary_diag['ICD9_CODE'].value_counts().head(20)

metadata = pd.read_csv("MIMIC III dataset HEF/extra_data/MIMIC_metadata_diagnose.csv")
top_20_df = top_20_primary.reset_index()
top_20_df.columns = ['ICD9_CODE', 'count']

top_20_with_names = top_20_df.merge(
    metadata[['ICD9_CODE', 'SHORT_DIAGNOSE']], 
    on='ICD9_CODE', 
    how='left'
)

print("\n=== Top 20 Primary Diagnoses with Descriptions ===\n")
print(top_20_with_names.to_string(index=False))

# Calculate coverage
total_primary = len(primary_diag)
top_20_coverage = (top_20_primary.sum() / total_primary) * 100
print(f"\nTop 20 codes cover {top_20_coverage:.1f}% of all primary diagnoses")




# Plot
plt.figure(figsize=(12, 8))
top_20_primary.sort_values().plot(kind='barh', color='coral', edgecolor='black')
plt.xlabel('Number of Patients', fontsize=12)
plt.ylabel('ICD9 Code', fontsize=12)
plt.title('Top 20 Most Common Primary Diagnoses', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


##### Observation: Primary Diagnoses

- **Top 20 codes** represent the most common critical care admission reasons
- These 20 codes account for **33%** of all primary diagnoses (check output)
- **Considerations for Model**
  - Common codes have reliable mortality statistics
  - Rare codes (appearing <10 times) will benefit from Bayesian smoothing (further supporting rationale to use Bayesian smoothing)
  - Will likely one-hot encode top N (TBD) primary diagnoses as additional features

#### 2.2.5 Diagnosis and Mortality Exploration

In [ ]:
train_with_counts = train_base_w_severity[['hadm_id', 'HOSPITAL_EXPIRE_FLAG', 'diagnosis_count']].copy()

# Bin diagnosis count
train_with_counts['diag_bin'] = pd.cut(
    train_with_counts['diagnosis_count'],
    bins=[0, 5, 10, 15, 20, 25, 40],
    labels=['1-5', '6-10', '11-15', '16-20', '21-25', '26+']
)

# Mortality rate by bin
mortality_by_count = (
    train_with_counts
    .groupby('diag_bin', observed=True)['HOSPITAL_EXPIRE_FLAG']
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'mortality_rate', 'count': 'n_patients'})
)

print("Mortality Rate by Diagnosis Count\n")
print(mortality_by_count)
print(f"\nOverall mortality rate: {train_base['HOSPITAL_EXPIRE_FLAG'].mean():.3f}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Mortality rate
axes[0].bar(mortality_by_count.index.astype(str), 
            mortality_by_count['mortality_rate'], 
            color='coral', edgecolor='black', alpha=0.8)
axes[0].set_xlabel('Number of Diagnoses', fontsize=12)
axes[0].set_ylabel('Mortality Rate', fontsize=12)
axes[0].set_title('Mortality Rate by Diagnosis Count', fontsize=13, fontweight='bold')
axes[0].axhline(train_base['HOSPITAL_EXPIRE_FLAG'].mean(), 
                color='blue', linestyle='--', linewidth=2, 
                label=f"Overall: {train_base['HOSPITAL_EXPIRE_FLAG'].mean():.3f}")
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Sample size
axes[1].bar(mortality_by_count.index.astype(str), 
            mortality_by_count['n_patients'], 
            color='steelblue', edgecolor='black', alpha=0.8)
axes[1].set_xlabel('Number of Diagnoses', fontsize=12)
axes[1].set_ylabel('Number of Patients', fontsize=12)
axes[1].set_title('Sample Size by Diagnosis Count', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


##### comment

- **Clear trend:** Mortality rate increases with diagnosis count
- **Low count (1-5):** Lowest mortaility rate at 4.1%
- **High count (26+):** Highest mortality rate at 18.7%
- **Interpretation:**
  - More diagnoses → more complex/severe medical conditions
  - `diagnosis_count` is a valuable predictor for mortality risk 
  - Very low counts may indicate coding issues or very short stays, but should stil watch out for few counts, but high severity diagnoses.


#### 2.2.6 Diagnosis Severity vs Mortaility



In [ ]:
# Pulling from function fit_diagnosis_features() created to pull diagnosis severity features into clean train data 
severity_cols = ['avg_diagnosis_severity', 'max_diagnosis_severity', 
                 'primary_diagnosis_severity']

# Check if severity features exist
if all(col in train_base_w_severity.columns for col in severity_cols):
    
    train_with_sev = train_base_w_severity[['HOSPITAL_EXPIRE_FLAG'] + severity_cols].copy()
    
    # Bin by avg severity quartiles
    train_with_sev['severity_quartile'] = pd.qcut(
        train_with_sev['avg_diagnosis_severity'],
        q=4,
        labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'],
        duplicates='drop'
    )
    
    mortality_by_sev = (
        train_with_sev
        .groupby('severity_quartile', observed=True)['HOSPITAL_EXPIRE_FLAG']
        .agg(['mean', 'count'])
        .rename(columns={'mean': 'mortality_rate', 'count': 'n_patients'})
    )
    
    print("Mortality Rate by Average Diagnosis Severity Quartile\n")
    print(mortality_by_sev)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Mortality by quartile
    axes[0].bar(mortality_by_sev.index.astype(str), 
                mortality_by_sev['mortality_rate'],
                color='darkred', edgecolor='black', alpha=0.8)
    axes[0].set_xlabel('Average Diagnosis Severity Quartile', fontsize=12)
    axes[0].set_ylabel('Mortality Rate', fontsize=12)
    axes[0].set_title('Mortality Rate by Diagnosis Severity', fontsize=13, fontweight='bold')
    axes[0].axhline(train_base['HOSPITAL_EXPIRE_FLAG'].mean(), 
                    color='blue', linestyle='--', linewidth=2,
                    label=f"Overall: {train_base['HOSPITAL_EXPIRE_FLAG'].mean():.3f}")
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Sample size
    axes[1].bar(mortality_by_sev.index.astype(str), 
                mortality_by_sev['n_patients'],
                color='steelblue', edgecolor='black', alpha=0.8)
    axes[1].set_xlabel('Average Diagnosis Severity Quartile', fontsize=12)
    axes[1].set_ylabel('Number of Patients', fontsize=12)
    axes[1].set_title('Sample Size by Severity Quartile', fontsize=13, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional: Show range of severity values
    print("\nDiagnosis Severity Feature Ranges\n")
    print(train_base_w_severity[severity_cols].describe())
    
else:
    print("Severity features not yet computed. Run fit_diagnosis_features() first.")

print(train_base_w_severity.shape)

### 3 Preprocessing 

#### 3.1 preprocessing functions

In [ ]:
# Creating pre-processing function to apply to both train and test data

def fit_preprocessing_pipeline(df, target_col='HOSPITAL_EXPIRE_FLAG'):
    """
    Fit all preprocessing transformers on training data.
    
    Parameters:
    -----------
    df : DataFrame
        Training data with target column
    target_col : str
        Name of target variable
        
    Returns:
    --------
    df_processed : DataFrame
        Fully preprocessed training data
    X_train : DataFrame
        Features only
    y_train : Series
        Target only
    state : dict
        Dictionary containing all fitted transformers
    """
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler
    import numpy as np
    
    df = df.copy()
    
    print("=== FITTING PREPROCESSING PIPELINE ===\n")
    
    # =============================
    # 1. IDENTIFY FEATURE TYPES
    # =============================
    
    vital_cols = [
        'HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean',
        'SysBP_Min', 'SysBP_Max', 'SysBP_Mean',
        'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
        'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean',
        'RespRate_Min', 'RespRate_Max', 'RespRate_Mean',
        'TempC_Min', 'TempC_Max', 'TempC_Mean',
        'SpO2_Min', 'SpO2_Max', 'SpO2_Mean',
        'Glucose_Min', 'Glucose_Max', 'Glucose_Mean'
    ]
    
    cat_cols = ['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 
                'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT']
    
    # Filter to existing columns
    vital_cols = [c for c in vital_cols if c in df.columns]
    cat_cols = [c for c in cat_cols if c in df.columns]
    
    print(f"Vitals to impute: {len(vital_cols)}")
    print(f"Categoricals to encode: {len(cat_cols)}")
    
    # =============================
    # 2. IMPUTE MISSING VALUES
    # =============================
    
    # Vitals: median
    imputer_vitals = SimpleImputer(strategy='median')
    df[vital_cols] = imputer_vitals.fit_transform(df[vital_cols])
    
    # Categoricals: mode
    imputer_cats = SimpleImputer(strategy='most_frequent')
    df[cat_cols] = imputer_cats.fit_transform(df[cat_cols])
    
    print(f"✓ Imputed vitals and categoricals")
    
    # =============================
    # 3. ONE-HOT ENCODE CATEGORICALS
    # =============================
    
    df_encoded = pd.get_dummies(
        df, 
        columns=cat_cols, 
        drop_first=True,
        dtype=int
    )
    
    # Save the columns after encoding for test set alignment
    encoded_columns = df_encoded.columns.tolist()
    
    print(f"✓ Encoded {len(cat_cols)} categoricals → {df_encoded.shape[1] - df.shape[1]} new features")
    
    # =============================
    # 4. SEPARATE FEATURES & TARGET
    # =============================
    
    y = df_encoded[target_col]
    X = df_encoded.drop(columns=[target_col])
    
    # =============================
    # 5. SCALE NUMERICAL FEATURES
    # =============================
    
    # Identify numerical columns (exclude binary dummies)
    num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
    binary_cols = [c for c in num_cols if X[c].nunique() == 2]
    num_to_scale = [c for c in num_cols if c not in binary_cols]
    
    print(f"✓ Scaling {len(num_to_scale)} numerical features (excluding {len(binary_cols)} binary)")
    
    scaler = StandardScaler()
    X[num_to_scale] = scaler.fit_transform(X[num_to_scale])
    
    # =============================
    # 6. FINAL VERIFICATION
    # =============================
    
    assert X.isnull().sum().sum() == 0, "Missing values remain!"
    assert len(X.select_dtypes(include=['object']).columns) == 0, "Non-numeric features remain!"
    assert not np.isinf(X.values).any(), "Infinite values found!"
    
    print(f"\n✓ PREPROCESSING COMPLETE")
    print(f"  Final shape: {X.shape}")
    print(f"  Target distribution: {y.value_counts(normalize=True).to_dict()}")
    
    # =============================
    # 7. PACK STATE FOR TEST SET
    # =============================
    
    state = {
        'imputer_vitals': imputer_vitals,
        'imputer_cats': imputer_cats,
        'scaler': scaler,
        'vital_cols': vital_cols,
        'cat_cols': cat_cols,
        'num_to_scale': num_to_scale,
        'encoded_columns': encoded_columns,
        'feature_columns': X.columns.tolist()  # Final feature order
    }
    
    return df_encoded, X, y, state


In [ ]:
# Apply Preprocessing to Test Set 

def apply_preprocessing_pipeline(df, state):
    """
    Apply fitted preprocessing transformers to new data (test set).
    
    Parameters:
    -----------
    df : DataFrame
        Test data (without target column)
    state : dict
        Dictionary of fitted transformers from fit_preprocessing_pipeline()
        
    Returns:
    --------
    X_test : DataFrame
        Preprocessed features, aligned with training features
    """
    import numpy as np
    
    df = df.copy()
    
    print("=== APPLYING PREPROCESSING PIPELINE ===\n")
    
    # =============================
    # 1. IMPUTE MISSING VALUES
    # =============================
    
    df[state['vital_cols']] = state['imputer_vitals'].transform(df[state['vital_cols']])
    df[state['cat_cols']] = state['imputer_cats'].transform(df[state['cat_cols']])
    
    print(f"✓ Imputed vitals and categoricals")
    
    # =============================
    # 2. ONE-HOT ENCODE CATEGORICALS
    # =============================
    
    df_encoded = pd.get_dummies(
        df,
        columns=state['cat_cols'],
        drop_first=True,
        dtype=int
    )
    
    print(f"✓ Encoded categoricals")
    
    # =============================
    # 3. ALIGN COLUMNS WITH TRAINING
    # =============================
    
    # Add missing columns (with 0s)
    missing_cols = set(state['feature_columns']) - set(df_encoded.columns)
    for col in missing_cols:
        df_encoded[col] = 0
    
    # Remove extra columns not in training
    extra_cols = set(df_encoded.columns) - set(state['feature_columns'])
    df_encoded = df_encoded.drop(columns=list(extra_cols))
    
    # Reorder to match training
    X = df_encoded[state['feature_columns']]
    
    print(f"✓ Aligned columns with training data")
    print(f"  Added {len(missing_cols)} missing columns")
    print(f"  Removed {len(extra_cols)} extra columns")
    
    # =============================
    # 4. SCALE NUMERICAL FEATURES
    # =============================
    
    X[state['num_to_scale']] = state['scaler'].transform(X[state['num_to_scale']])
    
    print(f"✓ Scaled {len(state['num_to_scale'])} numerical features")
    
    # =============================
    # 5. FINAL VERIFICATION
    # =============================
    
    assert X.shape[1] == len(state['feature_columns']), "Column mismatch!"
    assert X.isnull().sum().sum() == 0, "Missing values remain!"
    assert not np.isinf(X.values).any(), "Infinite values found!"
    
    print(f"\n✓ TEST PREPROCESSING COMPLETE")
    print(f"  Final shape: {X.shape}")
    
    return X


In [ ]:
# === CELL: Preprocess Training Data ===

# Fit pipeline on training data
train_processed, X_train, y_train, preprocess_state = fit_preprocessing_pipeline(
    train_base_w_severity,target_col='HOSPITAL_EXPIRE_FLAG'
)

# Save state for test set
import joblib
joblib.dump(preprocess_state, 'preprocessing_state.pkl')
print("\n✓ Saved preprocessing state for test set")

# Quick check
print(f"\nReady for modeling:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")


### Test file Preprocessing

In [ ]:
# Load test data
test = pd.read_csv('test.csv')

# Apply diagnosis features firsts
test_with_diag = apply_diagnosis_features(test, MIMIC_diagnoses, diag_state)

# Apply preprocessing pipeline
X_test = apply_preprocessing_pipeline(test_with_diag, preprocess_state)

print(f"\nTest data ready for predictions:")
print(f"  X_test: {X_test.shape}")

# Verify alignment
assert X_test.shape[1] == X_train.shape[1], "Feature count mismatch!"
print("✓ Test features perfectly aligned with training features")
